<a href="https://colab.research.google.com/github/ademozyavas/oct18foss/blob/master/mudek.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import shutil
import re
from openpyxl import load_workbook, Workbook
from openpyxl.chart import BarChart, Reference


#course based program outcome indices (column Number: course Outcome)
progOutcomeIndices = {2:'PÇ1',3:'PÇ2',4:'PÇ3',5:'PÇ4',6:'PÇ5',7:'PÇ6',8:'PÇ7',9:'PÇ8',
10:'PÇ9',11:'PÇ10',12:'PÇ11'}
#program indices for any course (studentID, Studentname, Course Outcomes)
anyCourseProgOutcomeIndices = {3:'PÇ1',4:'PÇ2',5:'PÇ3',6:'PÇ4',7:'PÇ5',8:'PÇ6',9:'PÇ7',10:'PÇ8',
11:'PÇ9',12:'PÇ10',13:'PÇ11'}
#for student based files
studentBasedProgOutcomeLetters = {'PÇ1':'B', 'PÇ2':'C','PÇ3':'D','PÇ4':'E','PÇ5':'F','PÇ6':'G', 'PÇ7':'H','PÇ8':'I',
'PÇ9':'J','PÇ10':'K','PÇ11':'L'}

fromPCToColIndexForACourse = {'PÇ1':3, 'PÇ2':4,'PÇ3':5,'PÇ4':6,'PÇ5':7,'PÇ6':8, 'PÇ7':9,'PÇ8':10,
'PÇ9':11,'PÇ10':12,'PÇ11':13}

courseCodeToFullName = {}
courseCodeToFullName['BIM101'] = 'BIM101 Programlamaya Giriş I'
courseCodeToFullName['BIM102'] = 'BIM102 Programlamaya Giriş II'
courseCodeToFullName['BIM103'] = 'BIM103 Bilgisayar Bilimlerine ve Mühendisliğine Giriş'
courseCodeToFullName['BIM104'] = 'BIM104 Bilgisayar Des. Teknik Resim'
courseCodeToFullName['FIZ101'] = 'FIZ101 Fizik I'
courseCodeToFullName['FIZ102'] = 'FIZ102 Fizik II'
courseCodeToFullName['ING101'] = 'ING101 İngilizce I'
courseCodeToFullName['ING102'] = 'ING102 İngilizce II'
courseCodeToFullName['KIM101'] = 'KIM101 Kimya'
courseCodeToFullName['MAT101'] = 'MAT101 Matematik I'
courseCodeToFullName['MAT102'] = 'MAT102 Matematik II'
courseCodeToFullName['MAT104'] = 'MAT104 Ayrık Matematik'
courseCodeToFullName['TRD101'] = 'TRD101 Türk Dili I'
courseCodeToFullName['TRD102'] = 'TRD102 Türk Dili II'

nonNumeralCellValues = ['kayıt sildi', 'dersi almadı','girmedi']


#read student Id - Name pairs from a file and return this as a dictionary
def readStudentIdNamePairs():
  os.chdir("/content/drive/MyDrive/SchoolChores/mudek")
  wb = load_workbook('studentIdNames.xlsx')
  sheet = wb.active
  studentIdNames = {}
  rowNo = 2
  
  while True:
    if sheet.cell(row=rowNo, column=1).value != None:
      studentIdNames[sheet.cell(row=rowNo,column=1).value] = sheet.cell(row=rowNo, column=2).value
    else:
      break
    rowNo += 1
  return studentIdNames


def writeToFileCourseBasedProgOutcomeAverages(courseIdToProgramOutcomeAverages):
  os.chdir("/content/drive/MyDrive/SchoolChores/mudek/genData")

  wb = Workbook()  
  sheet = wb.active 
  
  sheet['A1'] = "Program Kazanımları"  
  sheet['B1'] = "PÇ1"
  sheet['C1'] = "PÇ2"
  sheet['D1'] = "PÇ3"
  sheet['E1'] = "PÇ4"
  sheet['F1'] = "PÇ5"
  sheet['G1'] = "PÇ6"
  sheet['H1'] = "PÇ7"
  sheet['I1'] = "PÇ8"
  sheet['J1'] = "PÇ9"
  sheet['K1'] = "PÇ10"
  sheet['L1'] = "PÇ11"
  sheet['A2'] = 'Dersler'

  rowIndex = 3
  cellIndex = 1
  for aCourseCode in courseCodeToFullName:
    sheet['A'+str(rowIndex)] = courseCodeToFullName[aCourseCode] #A1 contains the course name
    for progOutcomeAverageForAll in courseIdToProgramOutcomeAverages[aCourseCode]:
      #column index is determined by the program outcome of the course
      sheet[studentBasedProgOutcomeLetters[progOutcomeAverageForAll[0]]+str(rowIndex)] = progOutcomeAverageForAll[1]
    rowIndex += 1
  wb.save("courseBasedProgOutcomeAverages.xlsx")





#Helper function used for debugging 
def printHelper(courseIdKey, listOfCoursesAndGrades): #coursesAndGradesDictForAStudent[courseIdKey]):
  print(courseIdKey)
  for aPair in listOfCoursesAndGrades:
    print(aPair)
  print("\n")

#creates a file with all courses and average of all grades for each outcome that
#class is associated with
#returned dictionary has courseID's as its keys and a list of programOutcome-Average 
#pairs
def createCourseBasedProgamOutcomeWithGrades():
  courseIdToProgramOutcomeSums = {}
  courseIdToProgramOutcomeAverages = {}
  courseIdToCount = {}
  #i = 0
  #print(len(students))
  #print(students), Students are fine up until now
  for aStudentIdKey in students:  #for every student
    #dictionary of courseID:list of pairs(programoutcome-grades dictionary
    coursesAndGradesDictForAStudent = students[aStudentIdKey]  
    # coursesAndGradesDictForAStudent has the following format (dictionary)
    #{'ING101': [('PÇ7', 76)], 'BIM101': [('PÇ2', 79)], 'BIM102': [('PÇ2', 80)], 
    #'BIM104': [('PÇ3', 64), ('PÇ6', 82), ('PÇ10', 80)], ...}
    for courseIdKey in coursesAndGradesDictForAStudent: #for every course of a student
      #if i == 0:
      #print(i)
      #print("\n")
      #printHelper(courseIdKey, coursesAndGradesDictForAStudent[courseIdKey])
      
      if not courseIdKey in courseIdToProgramOutcomeSums:
        courseIdToProgramOutcomeSums[courseIdKey] = coursesAndGradesDictForAStudent[courseIdKey] 
        #print(courseIdToProgramOutcomeSums)
      else:  #add each course with its program outcome and the student's grade to existing key (courseIdKey)
        #programOutcomeAndGradesForACourse is a list of pairs (programOutcome,StudentGrade)
        newProgramOutcomeAndGradesForACourse = coursesAndGradesDictForAStudent[courseIdKey]
        #print(programOutcomeAndGradesForACourse)
        #[('PÇ7', 62)]
        #...
        #[('PÇ3', 50), ('PÇ6', 38), ('PÇ10', 0)]
        # programOutcomeAndGradesForACourse has the above form
        #newCourseIdToProgOutcomesDict = {}
        newListOfPairs = []
        #for all pairs (tuples) of a course identified by courseIdKey
        for aNewProgramOutcomeAndGradePair in newProgramOutcomeAndGradesForACourse:
          #aProgramOutcomeAndGradePair is a tuple such as ('PÇ3', 60)
          newProgramOutcome = aNewProgramOutcomeAndGradePair[0]
          newGrade = aNewProgramOutcomeAndGradePair[1]
          pairsListSoFar = courseIdToProgramOutcomeSums[courseIdKey]
          #print(pairsListSoFar)
          runningSum = 0
          for aPairSoFar in pairsListSoFar:
            if aPairSoFar[0] == newProgramOutcome:
              runningSum += (int(aPairSoFar[1]) + int(newGrade))
              newListOfPairs.append((aPairSoFar[0],runningSum))
              break
        courseIdToProgramOutcomeSums[courseIdKey] = newListOfPairs
      
      if not courseIdKey in courseIdToCount:
        courseIdToCount[courseIdKey] = 1
      else:
        courseIdToCount[courseIdKey] = courseIdToCount[courseIdKey] + 1
  
  #courseIdToProgramOutcomeSums contains something like {'ING101': [('PÇ7', 1733)], 
  #'MAT101': [('PÇ1', 1688)], 'MAT104': [('PÇ1', 1227)], ...}
  #print("here")
  #print(courseIdToProgramOutcomeSums)

  for courseIdKey in courseIdToProgramOutcomeSums:  #for every course
    #print("*****")
    #print(courseIdToProgramOutcomeSums[courseIdKey])
    #input('courseIdToProgramOutcomeSums wait here')
    programOutcomeSumPairsList = courseIdToProgramOutcomeSums[courseIdKey]
    #print(type( programOutcomeSumPairsList))
    #input("type should be a list")
    programOutcomeAveragePairsList = []
    for programOutcomeSumTuple in programOutcomeSumPairsList: #for every tuple 
      #for programOutcomeSumPair in programOutcomeSumPairsList:
      #print(programOutcomeSumPair)
      #print(programOutcomeSumPair[0],programOutcomeSumPair[1])
      #val = input('wait here')
      programOutcomeAveragePairsList.append((programOutcomeSumTuple[0],int(programOutcomeSumTuple[1])/courseIdToCount[courseIdKey]))
    courseIdToProgramOutcomeAverages[courseIdKey] = programOutcomeAveragePairsList
  #print("there")
  return courseIdToProgramOutcomeAverages

#creates a file for each student with that student's all grades for all related
#program outcomes 
#for all courses
def createStudentFile(studentName, gradesDict):
  os.chdir("/content/drive/MyDrive/SchoolChores/mudek/genData")

  wb = Workbook()  
  sheet = wb.active 
  
  sheet['A1'] = "Program Kazanımları"  
  sheet['B1'] = "PÇ1"
  sheet['C1'] = "PÇ2"
  sheet['D1'] = "PÇ3"
  sheet['E1'] = "PÇ4"
  sheet['F1'] = "PÇ5"
  sheet['G1'] = "PÇ6"
  sheet['H1'] = "PÇ7"
  sheet['I1'] = "PÇ8"
  sheet['J1'] = "PÇ9"
  sheet['K1'] = "PÇ10"
  sheet['L1'] = "PÇ11"
  sheet['A2'] = 'Dersler'

  rowIndex = 3
  cellIndex = 1
  for aCourse in gradesDict:
    sheet['A'+str(rowIndex)] = courseCodeToFullName[aCourse] #A1 contains the course name
    for progOutcomeGrade in gradesDict[aCourse]:
      #column index is determined by the program outcome of the course
      sheet[studentBasedProgOutcomeLetters[progOutcomeGrade[0]]+str(rowIndex)] = progOutcomeGrade[1]
    rowIndex += 1
  wb.save(studentName+".xlsx")

#reads the file that contains all courses with their respected outcomes stored
#in a dictionary and returns it
# {...,'BIM104': [('PÇ3', 'Final notu'), ('PÇ6', 'Vize notu'), ('PÇ10', 'Ödev')], 'BIM201': [],...}
#is the format of the dictionary
def process_course_and_outcomes():
  os.chdir("/content/drive/MyDrive/SchoolChores/mudek")
  wb = load_workbook('courseBasedProgOutcomes.xlsx')
  sheet = wb.active
  rowNo = 3
  courses_and_outcomes = {}
  while True:
    if sheet.cell(row=rowNo, column=1).value != None:
      progOutcomes = []
      for colNo in range(2,13):
        if sheet.cell(row=rowNo, column=colNo).value != None:
          progOutcomes.append((progOutcomeIndices[colNo],sheet.cell(row=rowNo,column=colNo).value))
      courseNo = (sheet.cell(row=rowNo,column=1).value)[:6]
      courses_and_outcomes[courseNo] = progOutcomes
    else:
      break
    rowNo += 1
  return courses_and_outcomes

def process_cell(value):
  value = str(value)
  value = value.replace(",", ".")
  if value in nonNumeralCellValues:
    value = 0
  else:
    value = value.split(".")[0]
    value = int(value)
  return value

def process_course(aCourse):
  os.chdir("/content/drive/MyDrive/SchoolChores/mudek/course_based") 
  wb = load_workbook(aCourse)
  lastIndex = aCourse.rindex(".")
  #fileNameNoExt is BIM101, for example
  fileNameNoExt = aCourse[:lastIndex]
  sheet = wb.active
  rowNo = 2 #first row is titles for student ID, Student Name, and program outcomes
  #curProgOutcome = ''
  progOutIndices = courses_and_outcomes[fileNameNoExt]
  #students is a 'global' dictionary that contains all course grades for all students

  while True:
    if sheet.cell(row=rowNo, column=1).value != None:
      #there is more students to process
      studentNo = int(sheet.cell(row=rowNo,column=1).value)
      
      #for all course outcomes
      listProgOutcomesAndGradesForACourse = []
      for elem in courses_and_outcomes[fileNameNoExt]:
        colIndex = fromPCToColIndexForACourse[elem[0]]
        value = process_cell(sheet.cell(row=rowNo, column=colIndex).value)
        listProgOutcomesAndGradesForACourse.append((elem[0],value))
      if not studentNo in students:  
        students[studentNo] = {fileNameNoExt:listProgOutcomesAndGradesForACourse}
      else:
        students[studentNo][fileNameNoExt] = listProgOutcomesAndGradesForACourse
    else:
      break
    rowNo += 1
  

#check which program outcomes needs to be looked at for a course
courses_and_outcomes = process_course_and_outcomes()
#print("courses and outcomes:")
#print(courses_and_outcomes)

os.chdir("/content/drive/MyDrive/SchoolChores/mudek/course_based")
courses = os.listdir()
#students is a dictionary with studentNo being the key and its value being another 
#dictionary with key being the courseid (BIM101) and a list pairs (tuples) 
#where the first element of the pair is the program outcome (pc2) and its
#second element the grade earned for that program outcome.
students = {}

#read student ID - Name pairs from dictionary
studentIdNames = readStudentIdNamePairs()

for course in courses:
  #print(course)
  process_course(course)

# students dictionary has the student ID as its key and another dictionary as its
#value that has the courseID as its key and a list of pairs (programOutcome:grade)
#for all courses for the student
#{200501028: {'ING101': [('PÇ7', 76)], 'MAT101': [('PÇ1', 45)], ...}, ...}
#print(students)

if os.path.exists("/content/drive/MyDrive/SchoolChores/mudek/genData"):
  shutil.rmtree("/content/drive/MyDrive/SchoolChores/mudek/genData")
os.mkdir("/content/drive/MyDrive/SchoolChores/mudek/genData")  #where generated data will be saved
#os.mkdir("/content/drive/MyDrive/SchoolChores/mudek/genData/spring2021")
#create Student files (one file for each student with all grades for that student)
for aStudentId in students:
  createStudentFile(studentIdNames[aStudentId], students[aStudentId])

#create a file with all courses with their program outcome grades average (of all
#student grades for that outcome)
courseIdToProgramOutcomeAverages = createCourseBasedProgamOutcomeWithGrades()
writeToFileCourseBasedProgOutcomeAverages(courseIdToProgramOutcomeAverages)
print('DONE!!!')

DONE!!!


The notebook section below creates a courses dictionary with course ID (BIM101 for example) and all its program outcomes as its key.

In [ ]:
import os
from openpyxl import load_workbook, Workbook

os.chdir("/content/drive/MyDrive/SchoolChores/mudek")
wb = load_workbook('courseBasedProgOutcomes.xlsx')
sheet = wb.active
rowNo = 3
courses = {}
while True:
    if sheet.cell(row=rowNo, column=1).value != None:
      progOutcomes = []
      for colNo in range(2,13):
        if sheet.cell(row=rowNo, column=colNo).value != None:
          #print(sheet.cell(row=rowNo, column=colNo).value)
          progOutcomes.append(sheet.cell(row=rowNo,column=colNo).value)
      courses[sheet.cell(row=rowNo,column=1).value] = progOutcomes
    else:
      break
    rowNo += 1

#print(courses)
#print(coursenames)
#print(len(coursenames))

The notebook section below creates an excel sheet with all students IDs and names. It reads all the students information from all course files and saves them.

In [ ]:
import os
from openpyxl import load_workbook, Workbook

os.chdir("/content/drive/MyDrive/SchoolChores/mudek/course_based")
courses = os.listdir()


def createStudentIdNames(courseFile):
  wb = load_workbook(courseFile)
  sheet = wb.active
  rowNo = 2
  while True:
    if sheet.cell(row=rowNo, column=1).value != None:
      #there is more students to process
      studentNo = int(sheet.cell(row=rowNo,column=1).value)
      studentName = sheet.cell(row=rowNo, column=2).value
      students[studentNo] = studentName
    else:
      break
    rowNo += 1

def writeStudentsToFile():
  os.chdir("/content/drive/MyDrive/SchoolChores/mudek")
  wb = Workbook()  
  sheet = wb.active 
  sheet['A1'] = "Student ID"  
  sheet['B1'] = "Student Name"  
  cellIndex = 2
  for studentId in students:
    sheet['A'+str(cellIndex)] = studentId
    sheet['B'+str(cellIndex)] = students[studentId] 
    cellIndex += 1
  wb.save("studentIdNames.xlsx")
  
students = {}
for courseFile in courses:
  createStudentIdNames(courseFile)

#print(students)
#print(len(students))
writeStudentsToFile()

{200501028: 'AHMET EMİN KOLBASAR', 200501013: 'BERKE YAYKIN', 200501003: 'ELİF DENİZ ÜÇEL', 200501018: 'ERAY YILDIZ', 200501001: 'ESİN IRMAK ÖZTÜRK', 200501026: 'ESRA BUDAK', 200501014: 'FATMANAZ USTA', 200501007: 'GÖKSU COŞKUNYÜREK', 200501009: 'HALİL İBRAHİM BERK', 200501004: 'HATİCE MERCAN', 200501015: 'İBRAHİM ARDA BEYAZTAŞ', 200501024: 'İLKCAN ÜSTOĞLU', 200501019: 'KADİR ÇAGDAŞ', 200501017: 'MEHMET CAN SEÇKİN', 200501012: 'MEHMET KULAK', 200501002: 'MEMİK AYDOĞAN', 200501016: 'MERT ALİ KURTARAN', 200501023: 'MUHAMMET BURAK BOYRAZ', 200501020: 'SULTAN SENA ELGÖRMÜŞ', 200501011: 'TALHA ENES CILIZ', 200501010: 'TUBA TUĞÇE KOÇAN', 200501021: 'UMUT AYDIN', 200501027: 'YUSUF KARAASLAN', 200501006: 'ZEYNEP ERDOĞAN', 200501005: 'ZÜLAL ERGENÇ'}
25
